In [1]:
import os
import os.path as osp
import sys
from itertools import combinations
sys.path.append(osp.abspath('..'))

import numpy as np
from tqdm import tqdm_notebook
from sklearn.preprocessing import normalize
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

import config
from datasets.gtzan import GTZAN_SPEC

%load_ext autoreload
%autoreload 2

In [2]:
# Random seeds
np.random.seed(1234)
MIN_SEGMENTS = 1

In [3]:
train_set = GTZAN_SPEC(phase='val', test_size=0.1, val_size=0.4,
                       min_segments=MIN_SEGMENTS, randomized=True)
test_set = GTZAN_SPEC(phase='test', test_size=0.1, val_size=0.4, min_segments=MIN_SEGMENTS)

print('Train:', len(train_set))
print('Test:', len(test_set))
print('Shape:', train_set[0][0].shape)

Train: 400
Test: 100
Shape: (3, 221, 51)


In [ ]:
# aggregate over all frames, calculate simple summary statistics
# such as average, max or standard deviation for each channel
x_train = np.empty((400 * MIN_SEGMENTS, 221 * 3))
y_train = np.empty((400 * MIN_SEGMENTS,))
x_test = np.empty((100 * MIN_SEGMENTS, 221 * 3))
y_test = np.empty((100 * MIN_SEGMENTS,))
 

def populate_features(src_dataset, dst_x, dst_y):
    for sample_idx, (x, y) in tqdm_notebook(enumerate(src_dataset), total=len(src_dataset)):
        for segment_idx, x_ in enumerate(x):
            mean = []
            max = []
            std = []
            mean = np.mean(x_, axis=1)
            max = np.max(x_, axis=1)
            std = np.std(x_, axis=1)
            idx = sample_idx * MIN_SEGMENTS + segment_idx
            dst_x[idx:idx+1] = np.hstack((mean, max, std))
            dst_y[idx] = y

populate_features(train_set, x_train, y_train)
populate_features(test_set, x_test, y_test)

In [ ]:
COL_SIZE = 221
cols_selections = tuple({
    'mean': np.arange(COL_SIZE),
    'max': np.arange(COL_SIZE, COL_SIZE * 2),
    'std': np.arange(COL_SIZE * 2, COL_SIZE * 3)
}.items())

NORMALIZE = False
for num_combs in range(1, 4):
    for cols in combinations(cols_selections, num_combs):
        names = [col[0] for col in cols]
        print('Features:', ' '.join(names))
        cols = [col[1] for col in cols]
        cols = np.hstack(cols)
        x_train_ = np.take(x_train, cols, axis=1)
        x_test_ = np.take(x_test, cols, axis=1)
        
        if NORMALIZE:
            x_train_ = normalize(x_train_)
            x_test_ = normalize(x_test_)

        params = {
            'C': [10**x for x in range(-3, 3)],
            #'gamma': [10**x for x in range(-8, 0)],
            'random_state': [1234],
        }

        rs = GridSearchCV(LinearSVC(), params, n_jobs=-1, cv=10, verbose=0)
        rs.fit(x_train_, y_train)
        # print(rs.best_score_)
        # print(rs.best_params_)

        svm = LinearSVC(**rs.best_params_)
        svm.fit(x_train_, y_train)

        
        print('Train score:', accuracy_score(y_train, svm.predict(x_train_)))
        print('Test score:', accuracy_score(y_test, svm.predict(x_test_)))